### 通过张正友标定法获取相机参数
#### 步骤
```
1.准备标定板：使用一个已知尺寸的标定板（通常是黑白棋盘格），并在标定板上标记出若干个特征点。
2.拍摄图像：将标定板放置在不同的位置姿态下拍摄多张图像，并记录每张图像对应的相机姿态（即相机位姿）。
3.提取角点：对每张图像进行角点提取，即找到标定板上的特征点在图像中的对应位置。
4.计算相机姿态：通过已知的标定板尺寸和提取的角点，计算每张图像对应的相机位姿。
5.标定相机：使用相机标定公式，计算出相机的内参矩阵和畸变系数。
6.评估标定结果：使用标定结果评估指标（如重投影误差）来评估标定结果的准确性。
需要注意的是，该方法需要至少使用10张以上的图像才能稳健地标定相机。此外，标定板的布局和大小也会对标定结果产生影响。
```
#### 注意
```
1.标定板的质量：标定板需要具有较高的精度和稳定性，以保证提取的角点准确无误。同时，在摄影过程中需要避免标定板发生变形或损坏。
2.角点提取算法：角点提取算法需要能够精确地检测到标定板上的特征点，并且对噪声和光照变化具有一定的鲁棒性。目前常用的角点提取算法包括Harris角点算法、Shi-Tomasi算法等。
3.图像采集：在采集图像时，应尽可能保持相机姿态的多样性和分布均匀性，以覆盖尽可能广泛的空间。
4.相机位姿的估计：相机位姿的估计需要具有一定的精度和可靠性。通常可以使用三维-二维点对求解相机姿态，也可以使用直接法等方法进行位姿估计。
5.重投影误差的评估：重投影误差是评估标定结果好坏的一个重要指标。在评估时，需要注意将误差控制在一定范围内，同时了解误差来源，以便对标定结果进行进一步优化。
6.畸变校正的效果：畸变是相机成像过程中不可避免的问题，需要进行畸变校正。在进行畸变校正时，需要注意调整参数以达到最佳校正效果，并对校正结果进行评估和验证。
```


In [25]:
# 导包
import os
import cv2
import numpy as np

In [26]:
# 定义棋盘格模板规格:只算内角点个数,不算最外面的一圈点
pattern_size = (11, 8)
# 定义每个棋盘格的物理尺寸（单位：毫米）
square_size = 40.0
# 图像所在文件夹的位置
original_images = './CalibrationPlate/original/'   # 原始图片保存位置
resize_images = './CalibrationPlate/resize/'   # 调整尺寸后的图像保存位置
corner_images = './CalibrationPlate/corner/'   # 显示角点的图像保存位置

In [27]:
"""_调整图片大小,防止图片过大引起崩溃_
Args:
    input (_string_): _输入图片的文件夹路径_
    output (_string_): _输出图片的文件夹路径_
    width (_int_): _输出图片宽度_
    height (_int_): _输出图片高度_
"""
def ResizeImage(input, output, width, height):
    Images = os.listdir(input)
    for fname in Images:
        image = cv2.imread(input + fname)
        out = cv2.resize(image, (width, height))
        cv2.imwrite(output + fname, out)

In [28]:
# 调整图片尺寸
new_width = 800
new_height = 600
ResizeImage(original_images, resize_images, new_width, new_height)

In [29]:
# 世界坐标系中的棋盘格点,例如(0,0,0),(2,0,0)...(8,5,0)，去掉Z坐标，记为二维矩阵
world_point = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
# 将世界坐标系建在标定板上，所有点的Z坐标全部为0，所以只需要赋值x和y
world_point[:, :2] = np.mgrid[0:pattern_size[0]*square_size:square_size,
                              0:pattern_size[1]*square_size:square_size].T.reshape(-1, 2)
# 储存棋盘格角点的世界坐标和图像坐标对
world_points = []  # 世界坐标系中的三维点
image_points = []  # 图像平面的二维点

In [30]:
"""
角点精准化迭代过程的终止条件:
第一项:表示迭代次数达到最大次数时停止迭代;
第二项:表示角点位置变化的最小值已经达到最小时停止迭代;
第三项和第四项：表示设置寻找亚像素角点的参数--采用的停止准则是最大循环次数30和最大误差容限0.001
"""
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,
            30, 0.001)

In [31]:
images = os.listdir(resize_images)   # 读入图像序列
index = 0
for fname in images:
    image = cv2.imread(resize_images + '/' + fname)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)   # RGB转灰度
    # 寻找棋盘格角点,存放角点于corners中
    # 如果找到足够点对,将其存储起来,ret为非零值
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
    # 检测到角点后,进行亚像素级别角点检测,更新角点

    if ret == True:
        index += 1
        # 输入图像gray;角点初始坐标corners;搜索窗口为2*winsize+1;表示窗口的最小(-1.-1)表示忽略;求角点的迭代终止条件
        cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        world_points.append(world_point)   # 世界坐标
        image_points.append(corners)  # 图像坐标
        cv2.drawChessboardCorners(image, pattern_size, corners, ret)
        cv2.imwrite(corner_images + '/corners_' + str(index) + '.jpg', image)
        cv2.waitKey(10)
    cv2.destroyAllWindows()

In [32]:
"""
输入:世界坐标系里的位置;像素坐标;图像的像素尺寸大小;
输出:
    ret: 重投影误差;
    mtx: 内参矩阵;
    dist: 畸变系数;
    rvecs: 旋转向量 (外参数);
    tvecs: 平移向量 (外参数);
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    world_points, image_points, gray.shape[::-1], None, None)

# 保存相机参数(内参矩阵、畸变参数、旋转向量、平移向量)
np.savez('./Parameter/豪威OV48B.npz', mtx=mtx, dist=dist,
         rvecs=rvecs, tvecs=tvecs)  # 分别使用mtx,dist,rvecs,tvecs命名数组

print("ret(重投影误差):", ret,
      "\n\nmtx(内参矩阵):\n", mtx,
      "\n\ndist(畸变参数):\n", dist,  # 5个畸变参数,(k1,k2,p1,p2,k3)
      "\n\nrvecs(旋转向量):\n", rvecs,
      "\n\ntvecs(平移向量):\n", tvecs
      )

ret(重投影误差): 0.2991705035712761 

mtx(内参矩阵):
 [[596.34925072   0.         400.35710334]
 [  0.         596.49697012 300.47081541]
 [  0.           0.           1.        ]] 

dist(畸变参数):
 [[ 9.86732100e-02 -4.94565440e-01 -4.51708736e-04  2.89052555e-04
   8.22459145e-01]] 

rvecs(旋转向量):
 (array([[-0.0483275 ],
       [-0.05556281],
       [ 0.00805537]]), array([[-0.12482361],
       [ 0.15589444],
       [-0.07529847]]), array([[-0.17208193],
       [ 0.10014288],
       [-0.37814094]]), array([[-0.12937114],
       [ 0.11474067],
       [ 0.48904626]]), array([[0.30262844],
       [0.16153906],
       [0.03016236]]), array([[0.01218194],
       [0.56805057],
       [0.0150598 ]]), array([[-0.02910297],
       [-0.39260678],
       [-0.02031562]]), array([[-0.27239019],
       [-0.36934551],
       [ 0.12117097]]), array([[ 0.22883962],
       [-0.30550396],
       [-0.12688241]]), array([[ 0.06883582],
       [-0.27973277],
       [-0.06456626]]), array([[-0.2665373 ],
       [-0.185

### 获取Aruco码中心点的像素坐标

#### 方法原理
```
检测所拍摄的含有ArUco标记的图像,识别出每个ArUco码标记的中心点,在获取四个角点的像素坐标后计算出中心点的像素坐标
```

####


In [ ]:
# 导包
import cv2
import numpy as np

In [ ]:
# 加载用于生成标记的字典
dictionary = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250)
# 使用默认值初始化检测器参数
parameters = cv2.aruco.DetectorParameters_create()

index=0


### 根据两幅图像和对应的四个点的三维世界坐标和二维像素坐标来求其他点的世界坐标
#### 要求已知
```
1.两幅图像上对应的四个点的世界坐标和像素坐标。
2.相机的内参矩阵和畸变系数。
```
#### 实现方法
```
使用立体视觉（stereo vision）的方法：首先计算两个相机的姿态（旋转和平移矩阵），然后使用这些矩阵和相机内参矩阵进行三角测量（triangulation）以获取其他点的世界坐标。
```

In [ ]:
# 导包
import cv2
import numpy as np

In [ ]:
# 使用PnP算法估计相机姿态
def estimate_camera_pose(world_coords, pixel_coords, camera_matrix, dist_coeffs):
    _, rvec, tvec, _ = cv2.solvePnPRansac(
        world_coords, pixel_coords, camera_matrix, dist_coeffs)
    return rvec, tvec

In [ ]:
# 三角测量计算世界坐标
def triangulate_points(pixel_coords1, pixel_coords2, camera_matrix, rvec1, tvec1, rvec2, tvec2):
    proj_matrix1 = np.dot(camera_matrix, np.hstack(
        (cv2.Rodrigues(rvec1)[0], tvec1)))
    proj_matrix2 = np.dot(camera_matrix, np.hstack(
        (cv2.Rodrigues(rvec2)[0], tvec2)))
    world_coords = cv2.triangulatePoints(
        proj_matrix1, proj_matrix2, pixel_coords1.T, pixel_coords2.T).T
    world_coords = (world_coords / world_coords[:, 3:])[:, :3]
    return world_coords

In [ ]:
# 两幅图像中对应的世界坐标和像素坐标
world_coords_list = [
    np.array([[-50, -50, 0], [-50, 50, 0], [50, -50, 0],
             [50, 50, 0]], dtype=np.float32),
    np.array([[-50, -50, 0], [-50, 50, 0], [50, -50, 0],
             [50, 50, 0]], dtype=np.float32)
]